In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from scipy.special import expit
from edward.models import Bernoulli, Normal, Binomial
from random import randint


plt.style.use('ggplot')

P=3
K=10
omega_true = np.random.normal(0,1,[P,K])

def generator(batch_size):  # the M and the N here are not right..  
    probs = expit(np.matmul(omega_true,omega_true.T))
    while True:
        p1 = [randint(0,P-1) for _ in range(batch_size)]
        p2 = p1 = [randint(0,P-1) for _ in range(batch_size)]
        click = probs[p1,p2]>np.random.beta(1,1, batch_size)
        yield (p1,p2,click)

  
A = tf.placeholder(tf.int32, [None])
S = tf.placeholder(tf.int32, [None])
omega = Normal(tf.zeros([P, K]),tf.ones([P, K]))
# this is equiv to matmul(omega,omega.T)[A,S] - check
log_prob=(tf.gather(omega, A, axis=0) * tf.transpose(tf.gather(tf.transpose(omega), S, axis=1)))

Y = Bernoulli(logits=tf.reduce_sum(log_prob,1))

/Users/d.rohde/tensorflow3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
with tf.variable_scope('q_omega'):
    q_omega = Normal(tf.get_variable('loc', [P, K]),
                           tf.nn.softplus(tf.get_variable('scale', [P, K]))) 

In [3]:
N=100000
M=100
n_batch = int(N / M)
n_epoch = 1

Y_ph = tf.placeholder(tf.int32, None)

inference = ed.KLqp({omega: q_omega}, data={Y: Y_ph})
inference.initialize(n_iter=n_batch * n_epoch, n_samples=5, scale={Y: N / M})
tf.global_variables_initializer().run()

In [4]:
g=generator(int(M))

for _ in range(inference.n_iter):
    a, s, y = next(g)
    info_dict = inference.update({A: a, S: s, Y_ph: y})
    inference.print_progress(info_dict)

1000/1000 [100%] ██████████████████████████████ Elapsed: 5s | Loss: 8297.650


In [7]:
omega_bar = q_omega.mean().eval()
expit(np.matmul(omega_bar,omega_bar.T))

array([[ 0.99988699,  0.53613091,  0.05101121],
       [ 0.53613091,  0.97353494,  0.2594445 ],
       [ 0.05101121,  0.2594445 ,  0.99873561]], dtype=float32)

In [8]:
expit(np.matmul(omega_true,omega_true.T))

array([[ 0.99953905,  0.78405245,  0.99740794],
       [ 0.78405245,  0.96909677,  0.54599712],
       [ 0.99740794,  0.54599712,  0.99999998]])

0.6899744811276125